In [1]:
!pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1283, in

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import torch
import Model_Import_6
from torch import optim
import torch.nn as nn
from torch import optim
import random
import numpy as np
import gc

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
sentence_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [5]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
6.43825664
6.421954048
0.016302592


In [6]:
def train_two_styles_w_dev(model, optimizer, pos_context_embeds_list, neg_context_embeds_list, logits_list, token_ids_list, style_list_train, dev_logits, dev_pos_context, dev_neg_context, dev_token_ids, dev_style, epochs, num_samples = 100):
    CELoss = nn.CrossEntropyLoss()
    total_count = 0
    for epoch in range(epochs):
        
        to_shuffle = list(zip(logits_list, token_ids_list, style_list_train))

        random.shuffle(to_shuffle)

        logits_list, token_ids_list, style_list_train = zip(*to_shuffle)

        model.train()
        ag_loss_epoch = 0
        epoch_count = 0
        for example in range(len(logits_list)):
            if style_list_train[example] == 0:
                context_embeds_list = neg_context_embeds_list
            else:
                context_embeds_list = pos_context_embeds_list
            random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
            stacked_context_sample = torch.stack(random_context_samples, dim = 0)
            # print(stacked_context_sample.shape)
            optimizer.zero_grad()
            network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
            if token_ids_list[example]['input_ids'].shape[1] == 1:
                print("ONE text id")
                continue
            shifted_network_output = network_output[..., :-1, :].contiguous()
            shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
            loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
            ag_loss_epoch += loss
            epoch_count += 1
            total_count += 1
            loss.backward()
            optimizer.step()
            if example%1000 == 0:
                print(".", end = "")
        model.eval()
        CELoss_dev = nn.CrossEntropyLoss()
        dev_loss_acum = 0
        for dev_example in range(len(dev_logits)):
            if style_list_train[dev_example] == 0:
                dev_context = dev_neg_context
            else:
                dev_context = dev_pos_context
            random_context_samples_dev = random.sample(dev_context, num_samples) # need to make sure dev samples are the same... !!!!
            stacked_context_sample_dev = torch.stack(random_context_samples_dev, dim = 0)
            
            dev_network_output = model(stacked_context_sample_dev.to(device), dev_logits[dev_example].to(device))
            
            shifted_network_output_dev = dev_network_output[..., :-1, :].contiguous()
            shifted_text_ids_dev = dev_token_ids[dev_example]['input_ids'][..., 1:].contiguous().to(device)
            dev_loss = CELoss_dev(shifted_network_output_dev.view(-1, shifted_network_output_dev.size(-1)), shifted_text_ids_dev.view(-1))
            dev_loss_acum += dev_loss.item()
        full_dev_loss = dev_loss_acum / len(dev_logits)
        
        print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
        print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
        print(f"DEV LOSS: {full_dev_loss}")
        print("----------------------------------------")

In [7]:
def truncate_str_for_sen_embed(text_list, max_len=384):
    shortened_texts = []
    for text in text_list:
        start = 0
        end = max_len
        while True:
            if len(text) <= max_len:
                shortened_texts.append(text)
                break
            shortened_texts.append(text[:max_len])
            text = text[max_len:]
            
    return shortened_texts

In [8]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')

In [9]:
gatsby_shortened = truncate_str_for_sen_embed([gatsby], max_len=384)

origin_shortened = truncate_str_for_sen_embed([origin], max_len=384)




In [10]:
gatsby_shortened_train, gatsby_shortened_test = train_test_split(gatsby_shortened, test_size=0.2, random_state=42)
origin_shortened_train, origin_shortened_test = train_test_split(origin_shortened, test_size=0.2, random_state=42)

In [11]:
corpus = gatsby_shortened_train + origin_shortened_train

vectorizer = CountVectorizer(ngram_range=(1,3))

tranformed_corpus = vectorizer.fit_transform(corpus)

test_corpus =  gatsby_shortened_test + origin_shortened_test

transformed_test = vectorizer.transform(test_corpus)

In [12]:
print(len(gatsby_shortened_train))
print(len(gatsby_shortened_test))
print(len(origin_shortened_train))
print(len(origin_shortened_test))

563
141
1864
467


In [13]:
labels = []
test_labels = []
for ex in range(len(corpus)):
    if ex <= 563:
        labels.append(0)
    else:
        labels.append(1)
print(len(corpus))
print(len(labels))
for ex in range(len(test_corpus)):
    if ex <= 141:
        test_labels.append(0)
    else:
        test_labels.append(1)
print(len(test_corpus))
print(len(test_labels))

2427
2427
608
608


In [14]:
clf = LogisticRegression(random_state=0).fit(tranformed_corpus, labels)

In [15]:
clf.score(transformed_test, test_labels)

0.993421052631579

In [117]:
def one_style_generate(prompt, tokenizer, SAT_model, GPT_transformer, context_to_sample, num_samples = 100, num_tokens_to_generate = 1, sen_to_generate = 10):
  #Put models in eval mode
    SAT_model.eval()
    GPT_transformer.eval()
    
    for sent in range(sen_to_generate):
        random_context_samples = random.sample(context_to_sample, num_samples)
        model_context_input =  torch.stack(random_context_samples, dim = 0).to(device)
        expierment_list = []
        sentences = []
        # tokenize prompt
        current_tokenization = tokenizer.encode(prompt, return_tensors = 'pt').to(device)

        for generation in range(num_tokens_to_generate):
            # put tokenized prompt through GPT_transformer to get GPT Logits
            GPT_logits = GPT_transformer(current_tokenization).last_hidden_state.squeeze()

            # put model_context_input and the GPT Logits into SAT model
            adjusted_output = SAT_model(model_context_input, GPT_logits) 

            # Funtional softmax the SAT model output
            SM_adjusted_output = torch.nn.functional.softmax(adjusted_output, dim = 1)

            # argmax to get predicted token
            # predicted_tokens = torch.argmax(SM_adjusted_output, dim =1)

            # get topk tokens
            top_predicted_tokens = torch.topk(SM_adjusted_output[-1], 3, dim =0).indices
            top_predicted_tokens_prob = torch.topk(SM_adjusted_output[-1], 3, dim =0).values
            top_predicted_tokens_numpy = top_predicted_tokens.cpu().detach().numpy()
            top_predicted_tokens_prob_numpy = top_predicted_tokens_prob.cpu().detach().numpy()
            token_prob_sum = np.sum(top_predicted_tokens_prob_numpy)
            token_distribution = top_predicted_tokens_prob_numpy/token_prob_sum
            
            predicted_token =  torch.from_numpy(np.array(np.random.choice(top_predicted_tokens_numpy, p = token_distribution))).to(device)
            
            # print(top_predicted_tokens)
             # tok_k_predicted_words = tokenizer.decode(top_predicted_tokens, skip_special_tokens=True)
#             decoded_tokens = []
#             for token_k in top_predicted_tokens:
#                 decoded_tokens.append(tokenizer.decode(token_k, skip_special_tokens=True))
                
#             top_predicted_tokens_prob_cpu = top_predicted_tokens_prob.cpu()
            del top_predicted_tokens_prob
            # expierment_list.append((top_predicted_tokens_prob_cpu, decoded_tokens))


                
            #dif way of getting top predicted
            # predicted_token = top_predicted_tokens[0]
            # print(current_tokenization[-1][0])
            # print(predicted_token)
            # if predicted_token == 247 or predicted_token == current_tokenization[-1][0]:
            if predicted_token == 247:
                print("WEIRD TOKEN PREDICTED")
                predicted_token = top_predicted_tokens[1]

            # print(torch.amax(SM_adjusted_output[-1]))

            # print(predicted_tokens[-1].unsqueeze(0).unsqueeze(0))
            # print(current_tokenization)

            current_tokenization = torch.cat((current_tokenization, predicted_token.unsqueeze(0).unsqueeze(0)), 1)
            del predicted_token
            gc.collect()
            # print(current_tokenization)

        # decode
        sentence = []
        for i, beam in enumerate(current_tokenization):
            # print(f"{i}: {tokenizer.decode(beam)}")
            # print(f"{i}: {current_tokenization}")
            #print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")
            sentence.append(tokenizer.decode(beam, skip_special_tokens=True))
        sentences.append(sentence)
        del current_tokenization
        del random_context_samples
        del model_context_input
        gc.collect()
        # for prediction in expierment_list:
        #     print(prediction)
        print(".")
    return sentences
    # for i, beam in enumerate(predicted_tokens):
    #     # if i == 0:
    #     #   continue
    #     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)} token_id: {beam}")


In [17]:
def make_corpora_and_split(style_1, style_2):
    truncated_style_1 = truncate_str_for_sen_embed([style_1], max_len=384)
    truncated_style_2 = truncate_str_for_sen_embed([style_2], max_len=384)
    truncated_style_1_train, truncated_style_1_test = train_test_split(truncated_style_1, test_size=0.2, random_state=42)
    truncated_style_2_train, truncated_style_2_test = train_test_split(truncated_style_2, test_size=0.2, random_state=42)
    
    return truncated_style_1_train, truncated_style_1_test, truncated_style_2_train, truncated_style_2_test

In [18]:
def make_vectorizer_and_labels(truncated_style_1_train, truncated_style_1_test, truncated_style_2_train, truncated_style_2_test):
    ## make vectorizer
    labels = []
    test_lables = []
    corpus = truncated_style_1_train + truncated_style_2_train

    vectorizer = CountVectorizer(ngram_range=(1,3))

    tranformed_corpus = vectorizer.fit_transform(corpus)

    test_corpus =  truncated_style_1_test + truncated_style_2_test

    transformed_test = vectorizer.transform(test_corpus)
    
    for ex in range(len(corpus)):
        if ex <= len(truncated_style_1_train):
            labels.append(0)
        else:
            labels.append(1)
    for ex in range(len(corpus)):
        if ex <= len(truncated_style_1_test):
            test_labels.append(0)
        else:
            test_labels.append(1)
    
    return tranformed_corpus, transformed_test, labels, test_labels
    

In [19]:
def make_context_embed(model, sentence_list):
    # sen_embeds = []
    # count = 0
    with torch.no_grad():
        # for sen in sentence_list:
        #     if len(sen) == 0:
        #         print("empty sentence")
        #         continue
        
        sen_embeds = model.encode(sentence_list)
        return sen_embeds

In [20]:
def make_logits(model, tokenizer, book_sentence_list):
    logits = []
    token_ids = []
    count = 0
    with torch.no_grad():
        for sen in book_sentence_list:
            if len(sen) == 0:
                print("empty sentence")
                continue
            gpt_tokenized = tokenizer(sen, return_tensors = 'pt', truncation=True).to(device)
            gpt_embed = model(**gpt_tokenized).last_hidden_state.squeeze()
            gpt_tokenized_cpu = gpt_tokenized.to('cpu')
            del gpt_tokenized
            token_ids.append(gpt_tokenized_cpu) ## on the cpu!!!!
            gpt_embed_cpu = gpt_embed.to('cpu')
            del gpt_embed
            logits.append(gpt_embed_cpu)
            if count%1000 == 0:
                # t = torch.cuda.get_device_properties(0).total_memory
                # r = torch.cuda.memory_reserved(0)
                # a = torch.cuda.memory_allocated(0)
                # f = r-a  # free inside reserved
                # print(t/1000000000)
                # print(r/1000000000)
                # print(a/1000000000)
                # print(f/1000000000)
                print(count)
            count += 1
            
        return logits, token_ids

In [21]:
sen_embeds = make_context_embed(sentence_model, corpus)
print(sen_embeds.shape)
# sen_embeds_list = sen_embeds.tolist()
dev_sen_embeds = make_context_embed(sentence_model, test_corpus)
# dev_sen_embeds_list = dev_sen_embeds.tolist()

(2427, 768)


In [22]:
sen_embeds_list = []
dev_sen_embeds_list = []
for embed in sen_embeds:
    sen_embeds_list.append(torch.from_numpy(embed))
for embed in dev_sen_embeds:
    dev_sen_embeds_list.append(torch.from_numpy(embed))


In [23]:
print(len(sen_embeds))
print(len(sen_embeds_list))
print(len(corpus))

2427
2427
2427


In [24]:
dev_sen_embeds = make_context_embed(sentence_model, test_corpus)

In [25]:
logits, token_ids = make_logits(Model_Import_6.head_transformer, Model_Import_6.tokenizer, corpus)

0
1000
2000


In [26]:
dev_logits, dev_token_ids = make_logits(Model_Import_6.head_transformer, Model_Import_6.tokenizer, test_corpus)

0


In [27]:
#model, optimizer, pos_context_embeds_list, neg_context_embeds_list, logits_list, token_ids_list, style_list_train, dev_logits, dev_pos_context, dev_neg_context, dev_token_ids, dev_style, epochs, num_samples = 100

In [28]:
C_gatsby_embeds = sen_embeds_list[:563]
C_origin_embeds = sen_embeds_list[563:]
print(len(C_gatsby_embeds))
print(len(C_origin_embeds))
C_gatsby_embeds_dev = dev_sen_embeds_list[:141]
C_origin_embeds_dev = dev_sen_embeds_list[141:]
print(len(C_gatsby_embeds_dev))
print(len(C_origin_embeds_dev))

563
1864
141
467


In [29]:
# context_gatsby_embeds = []
# context_origin_embeds = []
# for embed in sen_embeds:
#     torch_embedsembeds
    

In [30]:
style_list_gatsby = []
style_list_origin = []
for embed in C_gatsby_embeds:
    style_list_gatsby.append(0)
for embed in C_origin_embeds:
    style_list_origin.append(1)
    
style_list_train = style_list_gatsby + style_list_origin

style_list_gatsby_dev = []
style_list_origin_dev = []
for embed in C_gatsby_embeds_dev:
    style_list_gatsby_dev.append(0)
for embed in C_origin_embeds_dev:
    style_list_origin_dev.append(1)
    
style_list_dev = style_list_gatsby_dev + style_list_origin_dev

In [31]:
num_heads = 8

In [32]:
print(len(C_gatsby_embeds))

563


In [33]:
print(logits[0].shape)

torch.Size([93, 1600])


In [34]:
print(C_gatsby_embeds[0].shape)

torch.Size([768])


In [127]:
pytorch_basic = Model_Import_6.MultiHeadModel_PyTorch_Stacked_Positional(C_gatsby_embeds[0].shape[0], logits[0].shape[1], heads = num_heads, attention_dim = int(logits[0].shape[1])).to(device) #
# pytorch_basic = Model_Import_6.MultiHeadModel_PyTorch_Positional(C_gatsby_embeds[0].shape[0], logits[0].shape[1], heads = num_heads, attention_dim = int(logits[0].shape[1])).to(device) #
# pytorch_basic = Model_Import_6.MultiHeadModel_PyTorch_Stacked_One_Alt(C_gatsby_embeds[0].shape[0], logits[0].shape[1], heads = num_heads, attention_dim = int(logits[0].shape[1])).to(device) #
# pytorch_basic = Model_Import_6.MultiHeadModel_PyTorch_Stacked(C_gatsby_embeds[0].shape[0], logits[0].shape[1], heads = num_heads, attention_dim = int(logits[0].shape[1])).to(device) #

# neg_optimizer = optim.Adam(pytorch_basic.parameters(), lr=0.00001,  weight_decay=0.001)
optimizer_basic = optim.RAdam(pytorch_basic.parameters(), lr=0.0001,  weight_decay=.0001) # could be useful transformers require warmup
# .0001 best WD so far lr=0.0001,  weight_decay=.0001

In [133]:
total_params = sum(
	param.numel() for param in pytorch_basic.parameters()
)
print(total_params)

trainable_params = sum(
	p.numel() for p in pytorch_basic.parameters() if p.requires_grad
)
print(total_params)

105838400
105838400


In [129]:
train_two_styles_w_dev(pytorch_basic, optimizer_basic, C_origin_embeds, C_gatsby_embeds, logits, token_ids, style_list_train, dev_logits, C_origin_embeds_dev, C_gatsby_embeds_dev, dev_token_ids, style_list_dev, 10, num_samples = 100)

...Epoch: 0, Epoch Examples: 2427
TRAIN LOSS: 2.916487216949463
DEV LOSS: 3.2152166148941768
----------------------------------------
...Epoch: 1, Epoch Examples: 2427
TRAIN LOSS: 2.6174700260162354
DEV LOSS: 3.2419174588040303
----------------------------------------
...Epoch: 2, Epoch Examples: 2427
TRAIN LOSS: 2.299574613571167
DEV LOSS: 3.3021455058141758
----------------------------------------
...Epoch: 3, Epoch Examples: 2427
TRAIN LOSS: 1.965745210647583
DEV LOSS: 3.372939638401333
----------------------------------------
...Epoch: 4, Epoch Examples: 2427
TRAIN LOSS: 1.622340440750122
DEV LOSS: 3.4998063225495186
----------------------------------------
...Epoch: 5, Epoch Examples: 2427
TRAIN LOSS: 1.2937655448913574
DEV LOSS: 3.650838586844896
----------------------------------------
...Epoch: 6, Epoch Examples: 2427
TRAIN LOSS: 0.9995564818382263
DEV LOSS: 3.7840108330312527
----------------------------------------
...Epoch: 7, Epoch Examples: 2427
TRAIN LOSS: 0.7638644576072

In [103]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
7.94820608
7.273941504
0.674264576


In [104]:
# prompt = "The"

In [105]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')

In [106]:
import nltk
gatsby_sen_list = nltk.sent_tokenize(gatsby)
origin_sen_list = nltk.sent_tokenize(origin)
print(len(gatsby_sen_list))
print(len(origin_sen_list))

2439
3981


In [107]:
# ref https://stackoverflow.com/questions/8689795/how-can-i-remove-non-ascii-characters-but-leave-periods-and-spaces
import re
import string
from collections import Counter
from collections import defaultdict
gatsby_first_words_list = []
for sen_g in gatsby_sen_list:
    sen_g = re.sub(r'^\s*|\s\s*', ' ', sen_g).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_g = filter(lambda x: x in printable, sen_g)
    sen_g = ''.join(filter(lambda x: x in printable, sen_g))
    sen_g_list = sen_g.split()
    #print(sen_g_list[0:3])
    gatsby_first_words_list.append(sen_g_list[0:3])
    
origin_first_words_list = []  
for sen_o in origin_sen_list:
    sen_o = re.sub(r'^\s*|\s\s*', ' ', sen_o).strip().rstrip().lstrip()
    printable = set(string.printable)
    sen_o = filter(lambda x: x in printable, sen_o)
    sen_o = ''.join(filter(lambda x: x in printable, sen_o))
    sen_o_list = sen_o.split()
    # print(sen_o_list[0:3])
    origin_first_words_list.append(sen_o_list[0:3]) 
    
initial_unigrams_gatsby = defaultdict(int)
initial_bigrams_gatsby = defaultdict(int)
initial_trigrams_gatsby = defaultdict(int)
initial_unigrams_origin = defaultdict(int)
initial_bigrams_origin = defaultdict(int)
initial_trigrams_origin = defaultdict(int)
for first_words in gatsby_first_words_list:
    initial_unigrams_gatsby[first_words[0]] += 1
    initial_bigrams_gatsby[' '.join(first_words[:2])] += 1
    initial_trigrams_gatsby[' '.join(first_words)] += 1
    
for first_words in origin_first_words_list:
    initial_unigrams_origin[first_words[0]] += 1
    initial_bigrams_origin[' '.join(first_words[:2])] += 1
    initial_trigrams_origin[' '.join(first_words)] += 1
    
unigram_overlap = set(initial_unigrams_gatsby.keys())
unigram_overlap = unigram_overlap.intersection(initial_unigrams_origin.keys())

bigram_overlap = set(initial_bigrams_gatsby.keys())
bigram_overlap = bigram_overlap.intersection(initial_bigrams_origin.keys())
   
trigram_overlap = set(initial_trigrams_gatsby.keys())
trigram_overlap = trigram_overlap.intersection(initial_trigrams_origin.keys())


# print(f"unigram_overlap: {str(unigram_overlap)}")
# print("\n")
print(f"bigram_overlap: {str(bigram_overlap)}")
print("\n")
# print(f"trigram_overlap: {str(trigram_overlap)}")

bigram_list = list(bigram_overlap)
    
# combine the list

# combined_g_o_list = gatsby_first_words_list + origin_first_words_list

# mini_sen_list = []
# for begin in combined_g_o_list:
#     mini_sen_list.append(' '.join(begin))
    
# # ref https://stackoverflow.com/questions/43473736/most-common-2-grams-using-python
# bigrams = zip(mini_sen_list, mini_sen_list[1:])
# counts = Counter(bigrams)
# print(counts.most_common())

bigram_overlap: {'And we', 'From the', 'Those who', 'We could', 'I see', 'At first', 'But I', 'Some little', 'There is', 'He must', 'I would', 'Do they', 'But no', 'They are', 'The masters', 'As soon', 'The eyes', 'And the', 'But with', 'It makes', 'He always', 'I will', 'Were all', 'He has', 'Of course', 'This is', 'When I', 'It was', 'At this', 'The day', 'To my', 'And what', 'For some', 'If there', 'Nothing at', 'After the', 'In fact,', 'How the', 'In two', 'But how', 'A man', 'By the', 'In my', 'Now I', 'Now, in', 'With the', 'A new', 'If it', 'For several', 'I am', 'The little', 'It seems', 'I could', 'I believe', 'Let the', 'But we', 'The three', 'All the', 'I remember', 'I was', 'And it', 'The old', 'A large', 'But it', 'And I', 'I think', 'The most', 'The truth', 'The whole', 'On the', 'A little', 'And when', 'The flowers', 'Now it', 'If I', 'After a', 'But the', 'Most of', 'Let us', 'So we', 'A few', 'I have', 'I should', 'And as', 'I know', 'It is', 'Not that', 'He would', 'A

In [108]:
lst = random.sample(bigram_list,10)

In [118]:
origins_SAT_generations = []
for prompt_ in lst:
    origins_SAT_generations.append(one_style_generate(prompt_, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, C_origin_embeds_dev, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 1)[0])

.
.
.
.
.
.
.
.
.
.


In [119]:
gatsby_SAT_generations = []
for prompt_ in lst:
    gatsby_SAT_generations.append(one_style_generate(prompt_, Model_Import_6.tokenizer, pytorch_basic, Model_Import_6.head_transformer, C_gatsby_embeds_dev, num_samples = 100, num_tokens_to_generate = 50, sen_to_generate = 1)[0])

.
.
.
WEIRD TOKEN PREDICTED
.
.
WEIRD TOKEN PREDICTED
.
.
.
.
.


In [120]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
7.94820608
7.273941504
0.674264576


In [121]:
def clean_strings(strings):
    cleaned_strings = []
    #print(strings[:20])
    for sen in strings:
        #print(sen)
        sen = re.sub(r"\\", "", sen[0])    # bizarrely, each string is inside of a list containing nothing but that string
        sen = re.sub(r"\n", " ", sen)
        sen = filter(lambda x: x in printable, sen)
        sen = ''.join(filter(lambda x: x in printable, sen))
        cleaned_strings.append(sen)
    #print(cleaned_strings[:20])
    
    return cleaned_strings

In [122]:
clean_gatsby_SAT_generations = clean_strings(gatsby_SAT_generations)
clean_origins_SAT_generations = clean_strings(origins_SAT_generations)

In [123]:
all_generations = clean_gatsby_SAT_generations + clean_origins_SAT_generations


generations_vectorized = vectorizer.transform(all_generations)

In [124]:
gatsby_gen_correct_labels = [0 for i in range(10)]
origin_gen_correct_labels = [1 for i in range(10)]

generation_labels = gatsby_gen_correct_labels + origin_gen_correct_labels

In [125]:
clf.score(generations_vectorized, generation_labels)

0.8

In [126]:
print(clean_gatsby_SAT_generations)

['But in hermaphrodite organisms which cross only one of the two sexes, that there are two sexes which cross. This case is sometimes called complement-giving. In the case of hermaphrodite animals which cross only once, that there', 'Let us all take the temperature of the house. Is it hot or cold, and if it is a shade of yellow or green, will it last? Is the colour pleasant and pleasing to our eyes? Is the room clean and neat? Has our dog left', 'I see. I see.  I broke away.  I see.  But I want to get one of those fancy dogs with fur between the legs before I do anything to it. I had them all lined up', 'This was my faultand it was on my head. But I had made that mistake before. It was in college. It was in a spring, when we were just starting out. Ill let you in on my little secret. We were', 'No one supposes that I am going to give up my right, as I have done, of saying that it is a mistake to see the whole organisation in the light of the principle of natural selection, that it is at

In [52]:
# all_seqs = []
# for seq in clean_gatsby_SAT_generations:
#     all_seqs.append(('GATSBY_SAT', seq))
# for seq in clean_origins_SAT_generations:
#     all_seqs.append(('ORIGIN_SAT', seq))

In [53]:
# import pandas as pd
# df = pd.DataFrame(all_seqs, columns = ['Source', 'Sequence'])

# df.to_csv('gatsby_origin_sat.csv', encoding='utf-8', index = False)